In [1]:
from google.colab import files
uploaded = files.upload()


Saving claims_5.csv to claims_5.csv


In [5]:
import pandas as pd
df = pd.read_csv("claims_5.csv", quotechar='"')
df

,ClaimID,ReportingDelay,AccidentHour,Age,WeeklyRate,Gender,MaritalStatus,HoursWorkedPerWeek,DependentChildren,DependentsOther,...,TotalDependents,EstimateGap,EstimateAccuracy,Severity,SeverityLabel,HasHighRiskWord,HasLowRiskWord,IncurredQuantile,DelayQuantile,EstimateCategory
0,1,15,11,28,500.00,M,S,44.0,0,0,...,0,-9348.745499,Overestimated,Low,Low,True,True,LowCost,ShortDelay,Overestimated
1,2,22,5,29,500.00,M,S,38.0,0,0,...,0,-11057.874976,Overestimated,Medium,Medium,False,True,MediumCost,ModerateDelay,Overestimated
2,3,22,7,28,197.37,M,M,16.0,0,0,...,0,-6505.509495,Overestimated,High,High,True,True,HighCost,ModerateDelay,Overestimated
3,4,15,12,40,0.00,M,M,0.0,1,0,...,1,-361.100000,Overestimated,Low,Low,True,True,LowCost,ShortDelay,Accurate
4,5,38,12,22,435.70,M,M,38.0,0,0,...,0,-23.840000,Overestimated,Medium,Medium,True,True,MediumCost,LongDelay,Accurate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21679,21687,23,9,53,200.00,M,M,38.0,0,0,...,0,-2067.833282,Overestimated,Medium,Medium,True,True,MediumCost,ModerateDelay,Overestimated
21680,21688,15,13,23,500.00,M,S,40.0,0,0,...,0,-91.364902,Overestimated,Medium,Medium,False,True,MediumCost,ShortDelay,Accurate
21681,21689,109,12,19,200.00,M,S,38.0,0,0,...,0,-753.750000,Overestimated,Low,Low,True,True,LowCost,LongDelay,Accurate
21682,21690,7,9,34,553.13,M,S,38.0,0,0,...,0,-738.375000,Overestimated,Medium,Medium,True,True,MediumCost,ShortDelay,Accurate


In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
# Encode target labels
label_encoder = LabelEncoder()
df["SeverityEncoded"] = label_encoder.fit_transform(df["SeverityLabel"])
num_labels = len(label_encoder.classes_)


In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["InputText"].tolist(),
    df["SeverityEncoded"].tolist(),
    test_size=0.2,
    stratify=df["SeverityEncoded"],
    random_state=42,
    shuffle=True
)


In [8]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)


In [9]:
import torch

class ClaimDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {"labels": torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.labels)

train_dataset = ClaimDataset(train_encodings, train_labels)
val_dataset = ClaimDataset(val_encodings, val_labels)


In [10]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from transformers import RobertaConfig, RobertaForSequenceClassification

# Include num_labels when creating the config
config = RobertaConfig.from_pretrained("roberta-base", num_labels=3, hidden_dropout_prob=0.3)

# Pass only the config to the model loader
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=config)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-severity-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,

    # Core training setup
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Simulate larger batch size
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Sequence length & scheduler
    lr_scheduler_type="cosine",
    warmup_steps=500,

    # Evaluation and saving
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # if using f1 metric
    greater_is_better=True
)


def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics
)

trainer.train()


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: a-imtiaz1663 (a-imtiaz1663-fast) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000300,0.000056,1.000000,1.000000,1.000000,1.000000
2,0.000100,0.000024,1.000000,1.000000,1.000000,1.000000
3,0.000100,0.000020,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=3255, training_loss=0.104213661721365, metrics={'train_runtime': 1305.1514, 'train_samples_per_second': 39.874, 'train_steps_per_second': 2.494, 'total_flos': 2941787872886940.0, 'train_loss': 0.104213661721365, 'epoch': 3.0})

In [11]:
# Create a directory to save the model
model_save_path = "roberta_severity_model"

# Save model and tokenizer
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"✅ RoBERTa model and tokenizer saved to: {model_save_path}")


✅ RoBERTa model and tokenizer saved to: roberta_severity_model


In [1]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Local directory where model was saved
model_save_path = "roberta_severity_model"

# Load tokenizer and model from local folder
tokenizer = RobertaTokenizer.from_pretrained(model_save_path)
model = RobertaForSequenceClassification.from_pretrained(model_save_path)

print("✅ Model and tokenizer loaded from local directory.")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\aimte\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\aimte\anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\aimte\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\aimte\anaconda\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\aimte\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\aimte\anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\aimte\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\aimte\anaconda\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



✅ Model and tokenizer loaded from local directory.


In [2]:
import torch
import numpy as np

def predict_severity(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    # Decode numeric prediction to label
    return label_encoder.inverse_transform([prediction])[0]


In [9]:
custom_text1 = "my eye got stabbed"
print(custom_text1)
print("Prediction:", predict_severity(custom_text1))
custom_text2 = "i am a 50 year old man. i had a car accident and fractured my right arm and leg."
print(custom_text2)
print("Prediction:", predict_severity(custom_text2))
custom_text3 = "my friend pulled my hair and ran away.it gave me a headache but i was fine"
print(custom_text3)
print("Prediction:", predict_severity(custom_text3))
custom_text4 = "i am 30 years old. i am a married woman. yesterday i fell off the stairs. the back of my head really hurts "
print(custom_text4)
print("Prediction:", predict_severity(custom_text4))



my eye got stabbed
Prediction: High
i am a 50 year old man. i had a car accident and fractured my right arm and leg.
Prediction: High
my friend pulled my hair and ran away.it gave me a headache but i was fine
Prediction: Low
i am 30 years old. i am a married woman. yesterday i fell off the stairs. the back of my head really hurts 
Prediction: High


In [10]:
custom_text5 = "my hand got stuck in the factory machinery, i took it out but my few fingers are injured"
print(custom_text5)
print("Prediction:", predict_severity(custom_text5))
custom_text6 = "my hand got stuck in the factory machinery, i took it out and its fine now"
print(custom_text6)
print("Prediction:", predict_severity(custom_text6))
custom_text7 = "my hand got stuck in the factory machinery,its bluish and swollen"
print(custom_text7)
print("Prediction:", predict_severity(custom_text7))


my hand got stuck in the factory machinery, i took it out but my few fingers are injured
Prediction: Medium
my hand got stuck in the factory machinery, i took it out and its fine now
Prediction: Low
my hand got stuck in the factory machinery,its bluish and swollen
Prediction: High


In [11]:
custom_text8 = "I am 18 years old. I slipped and hurt my foot but im fine now"
print(custom_text8)
print("Prediction:", predict_severity(custom_text8))
custom_text9 = "I am 30 years old. I slipped and hurt my foot and it slightly hurts"
print(custom_text9)
print("Prediction:", predict_severity(custom_text9))
custom_text10 = "I am 58 years old. I slipped and hurt my foot and it really hurts"
print(custom_text10)
print("Prediction:", predict_severity(custom_text10))


I am 18 years old. I slipped and hurt my foot but im fine now
Prediction: Low
I am 30 years old. I slipped and hurt my foot and it slightly hurts
Prediction: Medium
I am 58 years old. I slipped and hurt my foot and it really hurts
Prediction: High


In [15]:
custom_text11 = "I am a cancer patient and i am having trouble breathing"
print(custom_text11)
print("Prediction:", predict_severity(custom_text11))
custom_text12 = "i got a heart attack"
print(custom_text12)
print("Prediction:", predict_severity(custom_text12))
custom_text13 = "I am a 29-year-old male working full-time. I usually work about 40 hours every week and I have no dependents. I’m single and currently live alone. The incident occurred on 17/08/2016, and I reported it the next day on Wed. While carrying a small box in the storeroom, I slightly twisted my wrist. It wasn’t too painful at first, just a mild strain, but I reported it to be on the safe side. I continued working for the rest of the day without much trouble. My weekly wage is around 500. I consider this a minor issue,but it did need to be documented. "
print(custom_text13)
print("Prediction:", predict_severity(custom_text13))


I am a cancer patient and i am having trouble breathing
Prediction: High
i got a heart attack
Prediction: High
I am a 29-year-old male working full-time. I usually work about 40 hours every week and I have no dependents. I’m single and currently live alone. The incident occurred on 17/08/2016, and I reported it the next day on Wed. While carrying a small box in the storeroom, I slightly twisted my wrist. It wasn’t too painful at first, just a mild strain, but I reported it to be on the safe side. I continued working for the rest of the day without much trouble. My weekly wage is around 500. I consider this a minor issue,but it did need to be documented. 
Prediction: Low
